<a href="https://colab.research.google.com/github/nicolas998/WMF/blob/ghos_topo/Ghost_using_EE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages

In [ ]:
!apt-get install -qq libgdal-dev libproj-dev
!pip install cartopy
!pip install netCDF4
!pip install rasterio
!pip install git+https://github.com/nicolas998/WMF.git@ghos_topo --upgrade
!pip install pysheds
!pip install geopandas 
!pip install geemap

In [ ]:
import pylab as pl 
from wmf import wmf 
import numpy as np 
import pandas as pd 
import os
import osgeo
from wmf import ghost_topo as go 
import ee
import folium
import geopandas as geop
import warnings
warnings.filterwarnings('ignore')

# Set up a watershed 

## Read and process DEM 

In [3]:
DEM, DIR,epsg  = wmf.dem_process('/content/drive/MyDrive/IIHR/2021_IWA/data/dem.tif', dxp = 96, noData = -9999)

1. DEM raster readed
2. DEM depressions filled
3. DEM corrected
4. DIR map derived
5. Finish!


## Extract watershed

In [4]:
st = wmf.Stream(352378,4605633, DEM, DIR)
cu = wmf.SimuBasin(340517,4589890, DEM, DIR, stream=st, threshold=50)

1


## Get ghost class and its elements

In [5]:
gp = go.ghost_preprocess(cu, path_dem='/content/drive/MyDrive/IIHR/2021_IWA/data/dem.tif', 
    seg_threshold=700,
    seg_point_distance=75)

corrected = gp.get_segments_topology(epsilon=0.1)
gp.get_mesh_river_points(clean_close_points=True,
                         min_river2river_distance=50)
gp.get_mesh_grid_points(mesh_spaces=5,
                        border_iter=4,
                        clean_with_river=True,
                        min_dem2river_distance=50)

gp.get_voronoi_polygons()
gp.define_polygons_topology()

# Use earth explorer functions to set up the soil properties

In [6]:
gp.write_mesh_file('/content/drive/MyDrive/IIHR/2021_IWA/data/mesh.mesh','/content/drive/MyDrive/IIHR/2021_IWA/data/mesh.shp')

In [24]:
#Gets the soil data
soils = go.get_soils_data()
gp.get_physical_prop(soils, prop_name='soils')
#Fill the geometries with null values.
gp.polygons_shp['soils'].fillna(gp.polygons_shp['soils'].value_counts().idxmax(), 
                                inplace = True)
#Make the variable an integer
gp.polygons_shp['soils'] = gp.polygons_shp['soils'].astype(int)

In [8]:
#Gets the land use data
land_use = go.get_land_use_data()
gp.get_physical_prop(land_use, prop_name='land', band = 'cropland')
#Make the vartiable and integer
gp.polygons_shp['land'] = gp.polygons_shp['land'].astype(int)


In [35]:
path = '/content/drive/MyDrive/IIHR/2021_IWA/data/example.att'
f = open(path, 'w')
f.write('INDEX SOIL LC METEO LAI SS LAKE CLOSE_SEG BCns\n')
for i in gp.polygons_shp.index:
  poly = gp.polygons_shp.loc[i,'polygon']
  soil = gp.polygons_shp.loc[i,'soils']
  land = gp.polygons_shp.loc[i,'land']
  sides = gp.polygons_topology[i][3]
  f.write('%d %d %d 1 1 0 0 1 ' % (poly,soil, land))
  for z in range(sides):
          f.write('0 ')
  f.write('\n')
f.close()

# Use of the properties function in google colab 

# Get Earth Engine data 

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

## Resample soil data to the polygons 

In [7]:
# Convert the boundary of the watershed to an ee feature 
cu.Save_Basin2Map('/content/drive/MyDrive/IIHR/2021_IWA/data/divisory.shp', EPSG=int(cu.epsg))
shapefile = geop.read_file('/content/drive/MyDrive/IIHR/2021_IWA/data/divisory.shp')
shapefile.to_crs(4326, inplace = True)
boundary = ee.Feature(eval(shapefile.iloc[0:1].to_json())['features'][0])
xc = shapefile.centroid.x[0]
yc = shapefile.centroid.y[0]

#Get the polygons in ee
gp.write_mesh_file('/content/drive/MyDrive/IIHR/2021_IWA/data/mesh.mesh','/content/drive/MyDrive/IIHR/2021_IWA/data/mesh.shp')
shapefile = geop.read_file('/content/drive/MyDrive/IIHR/2021_IWA/data/mesh.shp')
shapefile.to_crs(4326, inplace = True)
features = []
for i in range(shapefile.shape[0]):
    geom = shapefile.iloc[i:i+1,:] 
    jsonDict = eval(geom.to_json()) 
    geojsonDict = jsonDict['features'][0] 
    features.append(ee.Feature(geojsonDict)) 
polygons = ee.FeatureCollection(features)

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:        
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:        
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'bands':['b10'],
  'min': 0,
  'max': 12,
  'palette': ["d5c36b","b96947","9d3706","ae868f","f86714","46d143",
    "368f20","3e5a14","ffd557","fff72e","ff5a9d","ff005b",]}

# Create a folium map object.
my_map = folium.Map(location=[yc, xc], zoom_start=10)

# Add the elevation model to the map object.
#my_map.add_ee_layer(data.updateMask(data.gt(0)), vis_params, 'soil_texture')
my_map.add_ee_layer(data_c, vis_params, 'soil_texture')
#my_map.add_ee_layer(fc.getInfo(), {}, 'divisory')
folium.GeoJson(
        data = boundary.getInfo(),
        name = 'divisory'
          ).add_to(my_map)
# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)